# Creating an Amundsen Sea Low (ASL) index

This work is a continuation of my 2013 and 2016 papers as described [here](https://scotthosking.com/asl_index)

## Read in data and run the calculations

In [4]:
from asli.asli import ASLICalculator
a = ASLICalculator(data_dir="../data/", 
                   mask_filename="era5_lsm.nc",
                   msl_pattern="ERA5/monthly/era5_mean_sea_level_pressure_monthly_1988.nc"
                   )
a.read_mask_data()
a.read_msl_data()

In [5]:
a.calculate(n_jobs=1)

  0%|          | 0/12 [00:01<?, ?it/s]


,time,lon,lat,ActCenPres,SectorPres,RelCenPres
0,1988-01-01,232.50,-69.00,986.125916,990.082153,-3.956238
1,1988-02-01,263.25,-68.25,976.671387,985.222778,-8.551392
2,1988-03-01,276.50,-66.50,978.056702,984.485535,-6.428833
3,1988-04-01,213.25,-72.00,968.617493,982.442200,-13.824707
4,1988-05-01,212.00,-63.75,971.383667,983.241638,-11.857971
5,1988-06-01,202.00,-66.00,979.515442,985.835693,-6.320251
6,1988-07-01,201.75,-77.50,974.239746,984.625366,-10.385620
7,1988-08-01,193.75,-73.75,969.384155,988.279724,-18.895569
8,1988-09-01,238.50,-71.00,975.072266,982.799683,-7.727417
9,1988-10-01,231.00,-69.75,980.524536,988.078186,-7.553650


## Plotting: location of minimas in pressure field

In [6]:
a.plot_region_all()

TypeError: ASLICalculator.plot_region_all() got an unexpected keyword argument 'cmap'